In [55]:
import sys
sys.path.append(".")
import unittest
import numpy as np
import matplotlib.pyplot as plt
from susie.transit_times import TransitTimes
from src.susie.ephemeris import Ephemeris, LinearModelEphemeris, QuadraticModelEphemeris, ModelEphemerisFactory
from scipy.optimize import curve_fit
test_epochs = np.array([0, 294, 298, 573])
test_mtts = np.array([0.0, 320.8780000000261, 325.24399999994785, 625.3850000002421])
test_mtts_err = np.array([0.00043, 0.00028, 0.00062, 0.00042])

test_P_linear = 1.0914223408652188 # period linear
test_P_err_linear = 9.998517417992763e-07 # period error linear
test_T0_linear =  -6.734666196939187e-05 # conjunction time
test_T0_err_linear = 0.0003502975050463415 # conjunction time error

test_P_quad = 1.0914215464474404 #period quad
test_P_err_quad = 9.150815726215122e-06 # period err quad
test_dPdE = 2.7744598987630543e-09#period change by epoch
test_dPdE_err = 3.188345582283935e-08#period change by epoch error
test_T0_quad = -1.415143555084551e-06 #conjunction time quad
test_T0_err_quad = 0.00042940561938685084#conjunction time err quad

test_observed_data = np.array([0.0, 320.8780000000261, 325.24399999994785, 625.3850000002421])
test_uncertainties= np.array([0.00043, 0.00028, 0.00062, 0.00042])



In [56]:
epoch_data = np.array([0, 294, 298, 573])
mid_transit_time_data = np.array([0.0, 320.8780000000261, 325.24399999994785, 625.3850000002421])
mid_transit_times_uncertainties_data = np.array([0.00043, 0.00028, 0.00062, 0.00042])
# STEP 2: Break data up into epochs, mid transit times, and error
epochs = epoch_data - np.min(epoch_data)
mid_transit_times = mid_transit_time_data - np.min(mid_transit_time_data)
mid_transit_times_err = mid_transit_times_uncertainties_data
# STEP 2.5 (Optional): Make sure the epochs are integers and not floats
epochs = epochs.astype('int')

In [57]:
transit_times_obj1 = TransitTimes('jd', epochs, mid_transit_times, mid_transit_times_err, time_scale='tdb')

In [58]:
ephemeris_obj1 = Ephemeris(transit_times_obj1)

In [59]:
linear_model_data = ephemeris_obj1.get_model_ephemeris('linear')
linear_model_data

{'period': 1.0914223408652188,
 'period_err': 9.998517417992763e-07,
 'conjunction_time': -6.734666196939187e-05,
 'conjunction_time_err': 0.0003502975050463415,
 'model_type': 'linear',
 'model_data': array([-6.73466620e-05,  3.20878101e+02,  3.25243790e+02,  6.25384934e+02])}

In [60]:
def linear_model(test_x, test_P_linear, test_T0_linear):
    """Linear model function."""
    return test_P_linear * test_x + test_T0_linear


popt, pcov = curve_fit(linear_model, test_epochs, test_mtts, sigma=test_mtts_err, absolute_sigma=True)
unc = np.sqrt(np.diag(pcov))
return_data = {
    'period': popt[0],
    'period_err': unc[0],
    'conjunction_time': popt[1],
    'conjunction_time_err': unc[1]
}
return_data

{'period': 1.0914223408652188,
 'period_err': 9.998517417992763e-07,
 'conjunction_time': -6.734666196939187e-05,
 'conjunction_time_err': 0.0003502975050463415}

In [62]:
def quad_model(test_x, test_dPdE, test_P_quad, test_T0_quad):
    return 0.5*test_dPdE*test_x*test_x + test_P_quad*test_x + test_T0_quad
popt, pcov = curve_fit(quad_model, test_epochs, test_mtts, sigma=test_mtts_err, absolute_sigma=True)
unc = np.sqrt(np.diag(pcov))
return_data = {
            'conjunction_time': popt[2],
            'conjunction_time_err': unc[2],
            'period': popt[1],
            'period_err': unc[1],
            'period_change_by_epoch': popt[0],
            'period_change_by_epoch_err': unc[0],
        }
return_data

{'conjunction_time': -1.415143555084551e-06,
 'conjunction_time_err': 0.00042940561938685084,
 'period': 1.0914215464474404,
 'period_err': 9.150815726215122e-06,
 'period_change_by_epoch': 2.7744598987630543e-09,
 'period_change_by_epoch_err': 3.188345582283935e-08}

In [54]:
quadratic_model_data = ephemeris_obj1.get_model_ephemeris('quadratic')
quadratic_model_data

{'conjunction_time': -1.415143555084551e-06,
 'conjunction_time_err': 0.00042940561938685084,
 'period': 1.0914215464474404,
 'period_err': 9.150815726215122e-06,
 'period_change_by_epoch': 2.7744598987630543e-09,
 'period_change_by_epoch_err': 3.188345582283935e-08,
 'model_type': 'quadratic',
 'model_data': array([-1.41514356e-06,  3.20878053e+02,  3.25243743e+02,  6.25385000e+02])}

In [24]:
##linear fit
P = test_P_linear
x = test_mtts
T0 = test_T0_linear
P*x + T0

array([-6.73466620e-05,  3.50213351e+02,  3.54978500e+02,  6.82559093e+02])

In [25]:
##quad fit
P = test_P_quad
T0 = test_T0_quad
dPdE = test_dPdE
0.5*dPdE*x*x + P*x + T0

array([-1.41514356e-06,  3.50213304e+02,  3.54978455e+02,  6.82559205e+02])

In [47]:
##linear model uncertainites
T0_err_l = test_T0_err_linear
P_err_l = test_P_err_linear
np.sqrt((T0_err_l**2) + ((np.array([0, 294, 298, 573])**2)*(P_err_l**2)))

array([0.0003503 , 0.00045729, 0.00045988, 0.00067152])

In [35]:
##quad model uncertainite
T0_err_q = test_T0_err_quad
P_err_q = test_P_err_quad
dPdE_err = test_dPdE_err
np.sqrt((T0_err_q**2) + (((np.array([0, 294, 298, 573]))**2)*(P_err_q**2)) + ((1/4)*((np.array([0, 294, 298, 573])**4)*(dPdE_err**2))))

array([0.00042941, 0.00305304, 0.00310238, 0.00742118])

In [30]:
## calc quad ephem
period_change_by_epoch = test_dPdE
epochs = test_epochs
period = test_P_quad
conjunction_time = test_T0_quad
((0.5*period_change_by_epoch*(epochs**2)) + (period*epochs) + conjunction_time)

array([-1.41514356e-06,  3.20878053e+02,  3.25243743e+02,  6.25385000e+02])

In [48]:
##calc lin ephem
period = test_P_linear
conjunction_time = test_T0_linear
((period*epochs) + conjunction_time)

array([-6.73466620e-05,  3.20878101e+02,  3.25243790e+02,  6.25384934e+02])

In [49]:
#calc chi linear
observed_data = np.array([0.0, 320.8780000000261, 325.24399999994785, 625.3850000002421])
model_data = [-6.73466620e-05, 3.20878101e+02, 3.25243790e+02, 6.25384934e+02]
uncertainties =  np.array([0.00043, 0.00028, 0.00062, 0.00042])
np.sum(((observed_data - model_data)/uncertainties)**2)

0.29406284565290114

In [43]:
#calc chi quad
observed_data = np.array([0.0, 320.8780000000261, 325.24399999994785, 625.3850000002421])
model_data = np.array([-1.41514356e-06,  3.20878053e+02,  3.25243743e+02,  6.25385000e+02])
uncertainties =  np.array([0.00043, 0.00028, 0.00062, 0.00042])
np.sum(((observed_data - model_data)/uncertainties)**2)

0.20766342879185204

In [50]:
## calc bic lin
chi_squared= 0.29406284565290114
num_params=2

chi_squared + (num_params*np.log(4))


3.0666515678926825

In [51]:
## calc bic quad
chi_squared= 0.20766342879185204
num_params=3

chi_squared + (num_params*np.log(4))

4.3665465121515235

In [52]:
linear_bic = 3.0666515678926825
quadratic_bic = 4.3665465121515235
linear_bic - quadratic_bic

-1.299894944258841